# Demonstrate observables

In [1]:
import mira
from mira.metamodel import Observable, SympyExprStr
from mira.modeling import ModelObservable
import sympy
import torch
from copy import deepcopy as _d
from mira.metamodel.ops import stratify
from sympytorch import SymPyModule
from pyciemss.PetriNetODE.interfaces import (
    load_petri_model,
    setup_petri_model,
    sample_petri,
    calibrate,
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model, 
)
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble
)
import pandas as pd
from pyciemss.utils.interface_utils import convert_to_output_format, csv_to_list, solutions_to_observations
from typing import Dict, List 

Solution = Dict[str, torch.tensor]

In [2]:
def get_sampled_value(name, model):
    trace_handler = pyro.poutine.trace(model)
    trace = trace_handler.get_trace()
    if name in trace.nodes and trace.nodes[name]['type'] == 'sample':
        return trace.nodes[name]['value']
    raise KeyError(f'{name} not in trace {trace.nodes}')



In [26]:
from sympy import symbols, exp
from sympy.printing.mathml import mathml

I, S, R, beta, gamma = symbols('infected_population susceptible_population immune_population beta gamma')

expr = I*S*beta/(S+I+R)
mathml_str = mathml(expr)

print(mathml_str)


<apply><divide/><apply><times/><ci>&#946;</ci><ci><mml:msub><mml:mi>infected</mml:mi><mml:mi>population</mml:mi></mml:msub></ci><ci><mml:msub><mml:mi>susceptible</mml:mi><mml:mi>population</mml:mi></mml:msub></ci></apply><apply><plus/><ci><mml:msub><mml:mi>immune</mml:mi><mml:mi>population</mml:mi></mml:msub></ci><ci><mml:msub><mml:mi>infected</mml:mi><mml:mi>population</mml:mi></mml:msub></ci><ci><mml:msub><mml:mi>susceptible</mml:mi><mml:mi>population</mml:mi></mml:msub></ci></apply></apply>


## Observe half the true population

In [27]:
from mira.examples.sir import sir_parameterized
tm = _d(sir_parameterized)
tm.initials
symbols = set(tm.initials)
expr = sympy.Add(*[sympy.Symbol(s) for s in symbols])
tm.observables = {'half_population': Observable(
        name='half_population',
        expression=SympyExprStr(expr/2))
    }

tm.observables['half_population'].expression.args[0]

immune_population/2 + infected_population/2 + susceptible_population/2

In [5]:
tm.initials

{'susceptible_population': Initial(concept=Concept(name='susceptible_population', display_name=None, description=None, identifiers={'ido': '0000514'}, context={}, units=None), value=1.0),
 'infected_population': Initial(concept=Concept(name='infected_population', display_name=None, description=None, identifiers={'ido': '0000511'}, context={}, units=None), value=2.0),
 'immune_population': Initial(concept=Concept(name='immune_population', display_name=None, description=None, identifiers={'ido': '0000592'}, context={}, units=None), value=3.0)}

## Compile the observable expression to pytorch

In [6]:
half_population = SymPyModule(expressions=[observable.expression.args[0] 
                                           for observable in tm.observables.values()])

## Expected observable value

In [7]:
expected_total_population = dict(
    infected_population=torch.tensor(1.0),
    immune_population=torch.tensor(0.0),
    susceptible_population=torch.tensor(100.0)
)

expected_half_population = half_population(**expected_total_population)                                
assert expected_half_population == torch.tensor([50.5])

## Generate samples from the template model

In [8]:
G = mira.modeling.Model(tm)
G.observables

{'half_population': <mira.modeling.ModelObservable at 0x14606da50>}

In [9]:
import torch
import pyro
num_samples = 2
data_path = 'sir_data.csv'
#sir_path = '../../test/models/AMR_examples/sir_typed.json'
timepoints = [0.1, 0.2, 0.3]
raw_sir = load_petri_model(tm, compile_observables_p=True)

In [10]:
def observation_model(solution: Solution, var_name: str) -> None:
    pass

In [11]:
raw_sir.G.variables

{('susceptible_population',
  ('identity', 'ido:0000514')): <mira.modeling.Variable at 0x14600f850>,
 ('infected_population',
  ('identity', 'ido:0000511')): <mira.modeling.Variable at 0x14600f280>,
 ('immune_population',
  ('identity', 'ido:0000592')): <mira.modeling.Variable at 0x14606e2f0>}

In [12]:
sir = setup_petri_model(raw_sir, 0.0, dict(susceptible_population=1000.0, infected_population=1.0, immune_population=0.0))
sir_samples = sample_petri(sir, timepoints , num_samples)
sir_samples


{'beta': tensor([0.0934, 0.1010]),
 'gamma': tensor([0.1993, 0.1878]),
 'immune_population_sol': tensor([[0.0198, 0.0394, 0.0589],
         [0.0187, 0.0372, 0.0556]]),
 'infected_population_sol': tensor([[0.9895, 0.9790, 0.9687],
         [0.9913, 0.9828, 0.9743]]),
 'susceptible_population_sol': tensor([[999.9907, 999.9815, 999.9724],
         [999.9899, 999.9799, 999.9699]])}

In [13]:
sir_sample_df = convert_to_output_format(sir_samples, timepoints)
sir_sample_df

,timepoint_id,sample_id,beta_param,gamma_param,immune_population_sol,infected_population_sol,susceptible_population_sol,timepoint_(unknown)
0,0,0,0.093445,0.199305,0.019825,0.989461,999.990723,0.1
1,1,0,0.093445,0.199305,0.039442,0.979032,999.981506,0.2
2,2,0,0.093445,0.199305,0.058851,0.968714,999.972351,0.3
3,0,1,0.101037,0.187823,0.018701,0.991349,999.989929,0.1
4,1,1,0.101037,0.187823,0.037240,0.982773,999.979919,0.2
5,2,1,0.101037,0.187823,0.055619,0.974270,999.969910,0.3


In [14]:
observations = solutions_to_observations(timepoints, sir_sample_df.set_index(['timepoint_id', 'sample_id']))
observations[0]

,,Timestep,immune_population,infected_population,susceptible_population
timepoint_id,sample_id,,,,
0,0,0.1,0.019825,0.989461,999.990723
1,0,0.2,0.039442,0.979032,999.981506
2,0,0.3,0.058851,0.968714,999.972351


In [15]:
observations[0].to_csv(data_path, index=False)
sir_data = csv_to_list(data_path)
for timepoint, data in sir_data:
    data['half_population'] = (
        data['immune_population'] + data['susceptible_population'] + data['infected_population'])/2
sir_data
    
                      

[(0.1,
  {'immune_population': 0.019825272262096405,
   'infected_population': 0.9894606471061707,
   'susceptible_population': 999.99072265625,
   'half_population': 500.50000428780913}),
 (0.2,
  {'immune_population': 0.03944159299135208,
   'infected_population': 0.9790322780609131,
   'susceptible_population': 999.9815063476562,
   'half_population': 500.49999010935426}),
 (0.3,
  {'immune_population': 0.05885117128491402,
   'infected_population': 0.9687137007713318,
   'susceptible_population': 999.9723510742188,
   'half_population': 500.4999579731375})]

In [16]:
get_sampled_value('infected_population_sol', sir)

tensor([])

In [17]:
inferred_parameters = calibrate(sir, sir_data, num_iterations=10)


immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, torch.Size([3])
immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
half_population, t

In [18]:
calibrated_samples = sample_petri(sir, timepoints, num_samples, inferred_parameters)

In [19]:
calibrated_samples

{'beta': tensor([0.1001, 0.1005]),
 'gamma': tensor([0.2043, 0.2074]),
 'immune_population_sol': tensor([[0.0203, 0.0404, 0.0603],
         [0.0206, 0.0410, 0.0612]]),
 'infected_population_sol': tensor([[0.9896, 0.9794, 0.9692],
         [0.9894, 0.9788, 0.9684]]),
 'susceptible_population_sol': tensor([[999.9901, 999.9801, 999.9704],
         [999.9900, 999.9800, 999.9703]])}

In [20]:
convert_to_output_format(calibrated_samples, timepoints)

,timepoint_id,sample_id,beta_param,gamma_param,immune_population_sol,infected_population_sol,susceptible_population_sol,timepoint_(unknown)
0,0,0,0.100149,0.204292,0.020323,0.989630,999.990051,0.1
1,1,0,0.100149,0.204292,0.040435,0.979367,999.980103,0.2
2,2,0,0.100149,0.204292,0.060339,0.969211,999.970398,0.3
3,0,1,0.100506,0.207430,0.020632,0.989355,999.989990,0.1
4,1,1,0.100506,0.207430,0.041045,0.978823,999.980042,0.2
5,2,1,0.100506,0.207430,0.061241,0.968402,999.970276,0.3


## Nonobservable data

In [21]:
sir_data = csv_to_list(data_path)                     
for timepoint, data in sir_data:
    data['definitely_not_half_population'] = (
        data['immune_population'] + data['susceptible_population'] + data['infected_population'])/2
sir_data
    

[(0.1,
  {'immune_population': 0.019825272262096405,
   'infected_population': 0.9894606471061707,
   'susceptible_population': 999.99072265625,
   'definitely_not_half_population': 500.50000428780913}),
 (0.2,
  {'immune_population': 0.03944159299135208,
   'infected_population': 0.9790322780609131,
   'susceptible_population': 999.9815063476562,
   'definitely_not_half_population': 500.49999010935426}),
 (0.3,
  {'immune_population': 0.05885117128491402,
   'infected_population': 0.9687137007713318,
   'susceptible_population': 999.9723510742188,
   'definitely_not_half_population': 500.4999579731375})]

In [28]:
try:
    inferred_parameters = calibrate(sir, sir_data, num_iterations=10)
except KeyError as k:
    print(k)

ERROR:root:
                ###############################

                There was an exception in pyciemss
                
                Error occured in function: calibrate_petri

                Function docs : 
    Use variational inference with a mean-field variational family to infer the parameters of the model.
    

                ################################
            
Traceback (most recent call last):
  File "/Users/zuck016/Projects/Proposals/ASKEM/build/clean-build/src/pyciemss/custom_decorators.py", line 9, in wrapped
    result = function(*args, **kwargs)
  File "/Users/zuck016/Projects/Proposals/ASKEM/build/clean-build/src/pyciemss/PetriNetODE/interfaces.py", line 709, in calibrate_petri
    loss = svi.step(method=method)
  File "/Users/zuck016/.pyenv/versions/clean-build/lib/python3.10/site-packages/pyro/infer/svi.py", line 145, in step
    loss = self.loss_and_grads(self.model, self.guide, *args, **kwargs)
  File "/Users/zuck016/.pyenv/versions/clean-buil

immune_population, torch.Size([3])
infected_population, torch.Size([3])
susceptible_population, torch.Size([3])
'definitely_not_half_population'


## 